In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:

from audio.redis import *
from audio.audio import *
#from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json

In [4]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'
import subprocess
from pydub import AudioSegment
import io
from time import sleep

In [5]:
redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [6]:
async def get_next_chunk_start(diarization_result, length):
    if len(diarization_result)>0:
        last_speech = diarization_result[-1]
        if round(last_speech['end']) == length:
            print('interrupted')
            return last_speech['conv_start']
        

        else:
            print('non-interrupted') 
            return last_speech['conv_end']



    else: return None

In [7]:
async def diarize(client_id, audio_name, shift):

    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    df = pd.DataFrame(diarization.data)
    if len(df)>0:
        df['silence'] = df['start']-df['end'].shift()
        df['speaker_change'] = df['speaker'] != df['speaker'].shift()
        df['len'] = df['end'] - df['start']
        df = df[df['len'] > 0.5]
        df['speaker_change'] = np.where(df['silence']>2,True,df['speaker_change'])

        df['speaker_change'] = df['speaker_change'].cumsum()
        df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'})
        df['conv_start'] = df['start'] + shift
        df['conv_end'] = df['end'] + shift
        return df.to_dict('records')
    else: return df

In [8]:

async def transcribe(diarization_result,connection_id,audio_name):
    redis_inner_client = await get_inner_redis()
    for segment in diarization_result:
        segment['connection_id'] = connection_id
        segment['name'] = str(uuid.uuid4())
        segment['audio_id'] = audio_name
        await redis_inner_client.lpush('TranscribeQueue', json.dumps(segment))
        done = await redis_inner_client.brpop(f'TranscribeReady:{segment["name"]}')
        print('done')
        transcription = Transcript(segment["name"],redis_inner_client)
        await transcription.get()
        segment['transcription'] = transcription.data
        await redis_inner_client.lpush(f'Segment:{connection_id}', json.dumps(segment))

In [9]:
await get_connections_df('initialFeed_audio',redis_stream_client)

,queue,length
0,initialFeed_audio:024c4576-5b39-443d-8fc6-0467...,13


In [10]:
#[await redis_stream_client.delete(c) for c in connection_ids]

In [11]:
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_id = connections[0].replace('initialFeed_audio:','')


In [12]:
#connection_id = 'david_audio'

In [13]:

path = f'/app/testdata/{connection_id}.webm'

In [14]:
start = 2000
length = 20


In [15]:
on_start = False

In [ ]:

on_start = True

In [16]:
import time

In [17]:
for _ in range(30):
    try:
        await writestream2file(connection_id,redis_stream_client)
        print(start)
        if on_start:  audio_slicer = await AudioSlicer.from_file(path,format = 'webm')
        else       :  audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,start+600000)

        slice_duration = audio_slicer.audio.duration_seconds

        if slice_duration > length:

            audio_data = await audio_slicer.export_data()
            audio_name = str(uuid.uuid4())
            audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
            await audio.save()

            print('waiting diarize')
            diarization_result = await diarize(client_id,audio_name,start)
            print('waiting transcribe')
            await transcribe(diarization_result,connection_id,audio_name)
            start_ = await get_next_chunk_start(diarization_result,slice_duration)
            start = start_ if start_ else start+length

            on_start = False

        else:
            time.sleep(5)
            pass
    except KeyboardInterrupt:
        pass

2000
None
waiting diarize
waiting transcribe
done
done
done
done
done
done
done
done
done
done
non-interrupted
2595.611205432937


In [35]:
try:
    await writestream2file(connection_id,redis_stream_client)
    print(start,end)
    if on_start: audio_slicer = await AudioSlicer.from_file(path,format = 'webm')
    else       :  audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,end)
    audio_data = await audio_slicer.export_data()
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()

    print('waiting diarize')
    diarization_result = await diarize(client_id,audio_name,start)
    print('waiting transcribe')
    await transcribe(diarization_result,connection_id,audio_name)
    start_ = await get_next_chunk_start(diarization_result,length)
    start = start_ if start_ else start+length

    end = start+audio_slicer.audio.duration_seconds+length if on_start else end+length
    on_start = False

except KeyboardInterrupt:
    pass

543.5568760611205 640.5031578947369
None
waiting diarize
waiting transcribe
done


In [18]:
while True:
    try:
        await writestream2file(connection_id,redis_stream_client)
        print(start,end)
        audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,end)
        audio_data = await audio_slicer.export_data()
        audio_name = str(uuid.uuid4())
        audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
        await audio.save()

        print('waiting diarize')
        diarization_result = await diarize(client_id,audio_name,start)
        print('waiting transcribe')
        await transcribe(diarization_result,connection_id,audio_name)
        start_ = await get_next_chunk_start(diarization_result,length)
        start = start_ if start_ else start+length

        if end>audio_slicer.audio.duration_seconds:
            print('stop')
            sleep(end-audio_slicer.audio.duration_seconds)
        end+=length

    except KeyboardInterrupt:
        pass
        

0 600
None
waiting diarize
waiting transcribe
lpush
done
lpush
done
lpush
done
lpush
done
lpush


In [21]:
audio_slicer.audio

In [36]:
start

44.855687606112056

In [37]:
end

80

In [40]:
audio_data = await audio_slicer.export_data()

None


In [43]:
type(audio_data)

bytes

In [44]:

audio_name = str(uuid.uuid4())
audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
await audio.save()

In [46]:

print('waiting diarize')
diarization_result = await diarize(client_id,audio_name,start)

waiting diarize


In [26]:
audio_slicer.audio

In [ ]:
import time

In [46]:
await redis_inner_client.lpush('TranscribeQueue', '')
await redis_inner_client.lpush('SentTime', str(time.time()))

1

In [47]:
#await redis_inner_client.delete('REceived')

In [55]:

sent = await redis_inner_client.rpop('SentTime')
sent


In [56]:

rec = await redis_inner_client.rpop('REceived')
rec

In [57]:
float(rec)-float(sent)

TypeError: float() argument must be a string or a number, not 'NoneType'

In [51]:
sent

'1706826535.4113765'

In [17]:
audio_slicer.audio

In [34]:
from datetime import datetime

timestamp = 1706809847.887154
dt_object = datetime.fromtimestamp(timestamp)

print(dt_object)

2024-02-01 17:50:47.887154
